In [1]:
!nvidia-smi

Sat Nov  5 08:45:23 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P0    27W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!wget https://raw.githubusercontent.com/mohitgupta-omg/Kaggle-SMS-Spam-Collection-Dataset-/master/spam.csv

--2022-11-05 08:45:23--  https://raw.githubusercontent.com/mohitgupta-omg/Kaggle-SMS-Spam-Collection-Dataset-/master/spam.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 503663 (492K) [text/plain]
Saving to: ‘spam.csv’

spam.csv            100%[===================>] 491.86K  --.-KB/s    in 0.006s  

2022-11-05 08:45:23 (86.7 MB/s) - ‘spam.csv’ saved [503663/503663]



In [3]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 28.8 MB/s 
     |████████████████████████████████| 163 kB 59.3 MB/s 
     |████████████████████████████████| 7.6 MB 76.5 MB/s 


In [15]:
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import regex as re
from tqdm import tqdm
from transformers import AutoTokenizer, TFAutoModel, AutoConfig

In [16]:
data = pd.read_csv("/content/spam.csv", encoding = "ISO-8859-1")

In [17]:
data.drop(["Unnamed: 2", "Unnamed: 3","Unnamed: 4"], axis=1, inplace=True)

In [18]:
data 

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will Ì_ b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [19]:
data['gt'] = data['v1'].map({'ham':0,'spam':1})
data['text'] = data.v2.copy()
data['text'] = data.text.apply(lambda x: x.lower())

In [20]:
data.head()

,v1,v2,gt,text
0,ham,"Go until jurong point, crazy.. Available only ...",0,"go until jurong point, crazy.. available only ..."
1,ham,Ok lar... Joking wif u oni...,0,ok lar... joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1,free entry in 2 a wkly comp to win fa cup fina...
3,ham,U dun say so early hor... U c already then say...,0,u dun say so early hor... u c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro...",0,"nah i don't think he goes to usf, he lives aro..."


In [24]:
X_train, X_test, y_train, y_test = train_test_split(data['text'], data["gt"], test_size=0.2, random_state=0)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.25, random_state=1)

In [25]:
sentences=data['text']
labels=data['gt']
len(sentences),len(labels)

(5572, 5572)

In [26]:
MAX_LENGTH = 128
NUM_CLASSES = len(data["gt"].unique())
MODEL_NAME = 'bert-base-uncased'
MODEL_CONFIG = AutoConfig.from_pretrained(MODEL_NAME)

In [27]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, do_lower_case=False, add_special_tokens=True,
                                                max_length=MAX_LENGTH, pad_to_max_length=True, use_fast=True)

In [28]:
def Tokenizer(sentences):
    input_ids=[]
    attention_masks=[]

    for sent in tqdm(sentences):
        bert_inp=tokenizer.encode_plus(sent, add_special_tokens = True ,max_length=MAX_LENGTH, pad_to_max_length = True, return_attention_mask = True)
        input_ids.append(bert_inp['input_ids'])
        attention_masks.append(bert_inp['attention_mask'])

    input_ids=np.asarray(input_ids)
    attention_masks=np.array(attention_masks)
    return input_ids, attention_masks

In [29]:
input_ids_train, attention_masks_train = Tokenizer(X_train)
input_ids_test, attention_masks_test = Tokenizer(X_test)
input_ids_valid, attention_masks_vaild = Tokenizer(X_valid)

  0%|          | 0/3342 [00:00<?, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2310: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
100%|██████████| 1115/1115 [00:00<00:00, 3921.69it/s]


In [30]:
def create_model(config=MODEL_CONFIG):
    config.output_hidden_states = False
    transformer_model = TFAutoModel.from_pretrained(MODEL_NAME, config=config)

    input_ids_in = tf.keras.layers.Input(shape=(MAX_LENGTH,), name='input_token', dtype='int32')
    input_masks_in = tf.keras.layers.Input(shape=(MAX_LENGTH,), name='masked_token', dtype='int32') 
    embedding_layer = transformer_model(input_ids_in, input_masks_in)[0][:, 0, :]

    X = tf.keras.layers.Dense(20, activation='relu')(embedding_layer)
    X = tf.keras.layers.Dropout(0.2)(X)

    X = tf.keras.layers.Dense(10, activation='relu')(X)
    X = tf.keras.layers.Dropout(0.2)(X)

    X = tf.keras.layers.Dense(1, activation='sigmoid')(X)
    model = tf.keras.Model(inputs=[input_ids_in, input_masks_in], outputs = X)
    return model

In [31]:
model = create_model()

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [32]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_token (InputLayer)       [(None, 128)]        0           []                               
                                                                                                  
 masked_token (InputLayer)      [(None, 128)]        0           []                               
                                                                                                  
 tf_bert_model_1 (TFBertModel)  TFBaseModelOutputWi  109482240   ['input_token[0][0]',            
                                thPoolingAndCrossAt               'masked_token[0][0]']           
                                tentions(last_hidde                                               
                                n_state=(None, 128,                                         

In [33]:
model.compile(loss="binary_crossentropy", 
              optimizer=tf.keras.optimizers.Adam(learning_rate=5e-6),
              metrics=["accuracy"])

In [34]:
mc = tf.keras.callbacks.ModelCheckpoint(filepath="./best_model.hdf5",
                                        monitor="val_accuracy",
                                        verbos=1,
                                        save_best_only=True,
                                        mode="max")
history = model.fit([input_ids_train, attention_masks_train], 
                    y_train,
                    batch_size=64, 
                    epochs=10,
                    callbacks=[mc],
                    validation_data=([input_ids_valid, attention_masks_vaild], y_valid)
                    )

Epoch 1/10


53/53 [==============================] - 112s 2s/step - loss: 0.4905 - accuracy: 0.7651 - val_loss: 0.2053 - val_accuracy: 0.9570
Epoch 2/10
53/53 [==============================] - 93s 2s/step - loss: 0.2347 - accuracy: 0.9063 - val_loss: 0.0882 - val_accuracy: 0.9839
Epoch 3/10
53/53 [==============================] - 93s 2s/step - loss: 0.1737 - accuracy: 0.9324 - val_loss: 0.0603 - val_accuracy: 0.9848
Epoch 4/10
53/53 [==============================] - 93s 2s/step - loss: 0.1408 - accuracy: 0.9405 - val_loss: 0.0437 - val_accuracy: 0.9892
Epoch 5/10
53/53 [==============================] - 93s 2s/step - loss: 0.1094 - accuracy: 0.9494 - val_loss: 0.0406 - val_accuracy: 0.9910
Epoch 6/10
53/53 [==============================] - 87s 2s/step - loss: 0.0894 - accuracy: 0.9560 - val_loss: 0.0501 - val_accuracy: 0.9883
Epoch 7/10
53/53 [==============================] - 87s 2s/step - loss: 0.0814 - accuracy: 0.9578 - val_loss: 0.0528 - val_accuracy: 0.9892
Epoch 8/10
53/53 [============

In [ ]:
model.evaluate([input_ids_test, attention_masks_test], y_test)

#Bài tập
1. Chạy mô hình bert-base-cased để xem kết quả thế nào
2. Thay Bert bằng xlnet-base-cased để chạy thử xem kết quả thế nào
3. Theo như bài tập số 2 ở những layers cuối hãy thay thành các FC layers có số units lần lượt là 50, 30, 10